# Creando máscaras

Vamos a crear máscaras a partir del marcado realizado en roboflow (puntos)

In [1]:
import numpy as np
import cv2
from pycocotools.coco import COCO

# Ruta al archivo de anotaciones COCO
ann_file_test = './Dataset/Segundoparcialv8icoco-mmdetection/test/_annotations.coco.json'
ann_file_train = './Dataset/Segundoparcialv8icoco-mmdetection/train/_annotations.coco.json'
ann_file_val = './Dataset/Segundoparcialv8icoco-mmdetection/valid/_annotations.coco.json'

# Inicializar el objeto COCO con el archivo de anotaciones
coco_test = COCO(ann_file_test)
coco_train = COCO(ann_file_train)
coco_val = COCO(ann_file_val)
#print("-------------------")
#print(coco.info())
#print("-------------------")

# Obtener los identificadores de las imágenes en el conjunto de datos
img_ids_test = coco_test.getImgIds()
print("Cant. images test: ", len(img_ids_test))
img_ids_train = coco_train.getImgIds()
print("Cant. images train: ", len(img_ids_train))
img_ids_val = coco_val.getImgIds()
print("Cant. images valid: ", len(img_ids_val))

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Cant. images test:  50
Cant. images train:  993
Cant. images valid:  50


### Crear máscaras con one-hot encoding

Vamos a crear máscaras a partir del marcado realizado en roboflow (puntos), pero en lugar de crear una máscara binaria, vamos a crear una máscara con one-hot encoding, es decir, una máscara con tantos canales como clases haya en el dataset. Esto nos permitirá entrenar la red con una sola salida, en lugar de una salida por clase.

En nuestro caso solo tenemos 3 clases (cuaderno, sodas y ruedas), pero tenemos que tomar en cuenta el background por lo que la máscara tendrá 4 canales.

Para esto creamos la funcion `create_one_hot_mask` que recibe como parámetros el path de coco, el id de la imagen y el número de clases.


In [45]:
def create_one_hot_mask(coco, img_id, num_classes):
    img_info = coco.loadImgs(img_id)[0]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)
    a, b, c = img_info['file_name'].split('.')[:3]
    name_mask = f"{a}.{b}.{c}.png"
    
    # Crear una lista de máscaras binarias para cada clase (incluyendo el fondo)
    masks = [np.zeros((img_info['height'], img_info['width']), dtype=np.uint8) for _ in range(num_classes)]
    
    # Rellenar las máscaras binarias con las regiones de interés de las anotaciones COCO
    for annotation in annotations:
        segmentations = annotation['segmentation']
        class_id = annotation['category_id']  # Identificador de clase
        
        for segmentation in segmentations:
            # Desempaquetar las tuplas de las coordenadas de segmentación
            points = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
            
            # Convertir la segmentación en un polígono y asignar 1 a las regiones de interés en la máscara
            cv2.fillPoly(masks[class_id - 1], [points], 1) # type:ignore class_id - 1 para ajustar el índice (comenzando desde 0)
    
    # Concatenar las máscaras en un solo array tridimensional (one-hot encoding)
    mask = np.stack(masks, axis=-1)
    
    # Agregar el fondo como la cuarta clase en el one-hot encoding (todos los canales son 0 para el fondo)
    mask[..., num_classes - 1] = np.logical_not(mask.any(axis=-1)).astype(np.uint8)
    
    return mask, name_mask

### Generar máscaras

Esta función recorre todas las imágenes del dataset y genera las máscaras con one-hot encoding. Las máscaras se guardan en su respectivas carpetas.

In [35]:
def generate_masks(num_classes):
    # crear mascara para test
    for img_id in img_ids_test:
        mask, name_mask = create_one_hot_mask(coco_test, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv8icoco-mmdetection/testmasks/' + name_mask, mask*255)
    # crear mascara para train
    for img_id in img_ids_train:
        mask, name_mask = create_one_hot_mask(coco_train, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv8icoco-mmdetection/trainmasks/' + name_mask, mask*255)
    # crear mascara para valid
    for img_id in img_ids_val:
        mask, name_mask = create_one_hot_mask(coco_val, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv8icoco-mmdetection/validmasks/' + name_mask, mask*255)

In [46]:
# generamos las mascaras
generate_masks(num_classes=4)
print("Done")

Done
